In [4]:
import os
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [5]:
directory = 'D:/question-answering-using-llm/data'   #keep multiple files (.txt, .pdf) in data folder.

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

ImportError: partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"